In [186]:
import json
import gbench
from gbench import util, report
from gbench.util import *

In [187]:
import plotly.express as px
import pandas as pd
import numpy as np

In [188]:
keys = ['name', 'real_time', 'cpu_time', 'AShr', 'Add', 'LShr', 'Mul', 'Nodes', 'SDiv', 'SRem', 'Shl', 'Sub', 'UDiv', 'URem', 'pop_count', 'V:AND', 'V:CELLS', 'V:MUX', 'V:NOT', 'V:OR', 'V:XOR']

In [218]:
def get_value(bench, key):
    try:
        value = bench[key]
    except KeyError:
        return np.nan

    try:
        return int(value)
    except ValueError:
        return value

def data_frame(data):
    d = dict()
    for key in keys:
        d[key] = [get_value(bench, key) for bench in data['benchmarks']]
    return pd.DataFrame(data=d)

def load_data(date, benchset):
    path = os.path.join(date, benchset + '.json')
    with open(path, 'r') as file:
        return data_frame(json.load(file))
    
def load(date):
    return load_data(date, 'noopt'), load_data(date, 'simplify'), load_data(date, 'conjure')

In [243]:
def scatter(lhs, rhs, view):
    lhs_data, lhs_name = lhs
    rhs_data, rhs_name = rhs
    df = pd.merge(lhs_data, rhs_data, how='inner', on='name', 
                  suffixes=["-" + lhs_name, "-" + rhs_name])

    labelx = view + "-" + lhs_name
    labely = view + "-" + rhs_name
    fig = px.scatter(df, x=labelx, y=labely, text='name')

    fig.update_layout(template='plotly_white', hovermode='x')
    
    maximal = max(df[labelx] + df[labely])
    fig.add_shape(
        type='line',
        x0=0,
        y0=0,
        x1=maximal,
        y1=maximal,
        xref='x',
        yref='y',
        line=dict(color='gray', width=1)
    )
    
    fig.update_traces(mode="markers")
    return fig

In [244]:
new_noopt, new_simpl, new_conju = load('23-02-07')
abc_noopt, abc_simpl, abc_conju = load('23-01-25')

In [245]:
# pd.merge(new_noopt, abc_noopt, how='inner', on='name')

In [246]:
scatter((new_noopt, "new-noopt"), (abc_noopt, "abc-noopt"), "Nodes")

In [299]:
scatter((new_noopt, "new-noopt"), (abc_noopt, "abc-noopt"), "V:AND")

In [248]:
scatter((new_noopt, "new-noopt"), (new_conju, "new-alu-synthesis"), "V:CELLS")

In [269]:
scatter((new_noopt, "new-noopt"), (new_conju, "new-alu-synthesis"), "Nodes")

In [292]:
scatter((new_noopt, "new-noopt"), (new_conju, "new-alu-synthesis"), "Mul")

In [278]:
def compare(lhs, lhs_suff, rhs, rhs_suff, prefixes):
    df = pd.merge(lhs, rhs, how='inner', on='name', suffixes=["-" + lhs_suff, "-" + rhs_suff])
    filtered = df.loc[:, [col for col in df.columns if any(col.startswith(prefix) for prefix in prefixes)]]
    filtered = filtered.reindex(sorted(filtered.columns), axis=1)
    return pd.concat([df['name'], filtered], axis = 1)

In [298]:
df = compare(new_noopt, "new", new_conju, "alu", ["Mul", "Add", "Nodes", "V:CELLS", "V:AND", "V:OR"])
df[df['name'].str.startswith("lift_source")]

,name,Add-alu,Add-new,Mul-alu,Mul-new,Nodes-alu,Nodes-new,V:AND-alu,V:AND-new,V:CELLS-alu,V:CELLS-new,V:OR-alu,V:OR-new
30,"lift_source/""min:small""",5,5,0,0,312,226,358,163,2786,2499,1012,933
31,"lift_source/""min:adds""",37,282,0,0,10382,1946,35453,16572,81976,52068,33894,11985
32,"lift_source/""min:mul-16""",11,34,1,4,935,423,4382,2753,12572,10424,4072,2584
33,"lift_source/""big:muls""",28,183,1,12,6192,1419,27952,31482,61631,77144,23025,15889
34,"lift_source/""min:idivs""",13,46,0,0,1437,658,32546,33475,80131,84395,20096,19519
35,"lift_source/""big:divs""",12,67,0,0,2772,1069,61821,62080,152532,156021,38893,36442
36,"lift_source/""min:subs""",39,269,0,0,10021,1936,34096,16438,77954,51288,32590,11851
37,"lift_source/""min:xors""",18,253,0,0,8888,1791,28709,15673,67498,50412,29005,11601
38,"lift_source/""big:3d_image_toolkit""",37,261,1,4,18505,3933,105760,76062,245424,196651,83784,46171
39,"lift_source/""big:router_simulator""",35,278,1,3,19109,3979,108811,63472,253054,166019,88512,39156
